<a href="https://colab.research.google.com/github/HUFSDoeun/datacampus_dao/blob/main/%ED%9A%A8%EC%9C%A8%EC%84%B1_%ED%8F%89%EA%B0%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지

In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 데이터 로드

In [ ]:
path = "/content/drive/MyDrive/데청캠2022/데청캠 - 프로젝트/"

In [ ]:
file_name_list = os.listdir(path)
file_name_list = [file for file in file_name_list if file.endswith(".csv")]

file_dict = {}
for file in file_name_list:
  file_dict[file[:-4]] = pd.read_csv(path+file, encoding = 'cp949')

# 효율성 평가
- [참고 논문 : 야생동물 이용빈도 및 종다양도를 활용한 생태통로 효율성 평가 - 국립생태원](https://koreascience.kr/article/JAKO201908551716737.pdf)

## 종 다양도 계산
- 종 다양도 = 이용 확인 종 수 / 주변 동물종 수

In [ ]:
for df_name, df in file_dict.items():
  df['종 다양도'] = df['이용확인종_수'] / df['주변 동물종 개수']
  df['종 다양도'] = df['종 다양도'].fillna(0)
  df.replace(np.inf, 0, inplace = True)

## 효율성 판단
- 효율성 양호 조건 : (이용빈도 >= mean) AND (종 다양도 >= 1)

In [ ]:
freq_mean_dict = {} # 이용빈도 평균

for df_name, df in file_dict.items():
  mean = df['일평균이용빈도'].mean()
  freq_mean_dict[df_name] = mean
  df['효율성평가'] = 0
  for idx in range(len(df)):
    if df.loc[idx, '일평균이용빈도'] >= mean and df.loc[idx, '종 다양도'] >= 1:
      df.loc[idx, '효율성평가'] = 1

In [ ]:
for df_name, df in file_dict.items():
  df.to_csv(path + df_name + '효율성.csv', index=False, encoding='cp949')